In [21]:
import os
import sys

os.chdir('/scratch/edk202/word2gm-fast/notebooks')
sys.path.append('../src/word2gm_fast')
sys.path.append('../')

import dataprep.load_and_filter_corpus
from tests.test_load_and_filter_corpus import TestLoadAndFilterCorpus

In [22]:
!python -m unittest test_load_and_filter_corpus

E
ERROR: test_load_and_filter_corpus (unittest.loader._FailedTest.test_load_and_filter_corpus)
----------------------------------------------------------------------
ImportError: Failed to import test module: test_load_and_filter_corpus
Traceback (most recent call last):
  File "/ext3/miniforge3/envs/word2gm-fast2/lib/python3.12/unittest/loader.py", line 137, in loadTestsFromName
    module = __import__(module_name)
             ^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'test_load_and_filter_corpus'


----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (errors=1)
